In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime
import json
from leagueDataParser import parseLeagueData
from leagueDataCleaner import cleanLeagueData

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [3]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100
    break

In [4]:
matchData = None

for j,i in enumerate(matchs):
    p = lol.match.by_id(region,i)
    p = parseLeagueData(p, puuid)
    if j == 0:
        matchData = pd.DataFrame(columns = list(p.keys()))
    matchData.loc[j,:] = p
    if j == 10:
        break
matchData

,gameDuration,gameMode,gameStartTimestamp,gameType,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,...,primaryVar23,primaryVar33,secondaryPerk0,secondaryVar10,secondaryVar20,secondaryVar30,secondaryPerk1,secondaryVar11,secondaryVar21,secondaryVar31
0,1012,CLASSIC,1683131536235,MATCHED_GAME,0,0,0,0,0,0,...,0,0,8304,12,0,0,8345,3,0,0
1,1632,CLASSIC,1683129617935,MATCHED_GAME,0,0,5,0,0,0,...,0,0,8304,9,4,5,8345,3,0,0
2,180,CLASSIC,1683129138161,MATCHED_GAME,0,0,0,0,0,0,...,0,0,8304,0,0,0,8345,1,0,0
3,2146,CLASSIC,1683053919462,MATCHED_GAME,0,0,7,0,0,0,...,0,0,8304,10,3,0,8345,3,0,0
4,2191,CLASSIC,1682968146514,MATCHED_GAME,0,0,8,0,0,0,...,0,0,8304,10,3,0,8345,3,0,0
5,1563,CLASSIC,1682966275969,MATCHED_GAME,0,0,2,0,0,0,...,0,0,8139,723,0,0,8135,320,4,0
6,1341,CLASSIC,1682964525299,MATCHED_GAME,0,0,2,0,0,0,...,0,0,8304,9,4,5,8345,3,0,0
7,1565,ARAM,1682959684663,MATCHED_GAME,0,0,19,0,0,0,...,0,0,8139,1448,0,0,8135,450,5,0
8,1193,CLASSIC,1682958141543,MATCHED_GAME,0,0,8,0,0,0,...,0,0,8304,9,0,0,8345,3,0,0
9,131,CLASSIC,1682955322452,MATCHED_GAME,0,0,0,0,0,0,...,0,0,8304,0,0,0,8345,1,0,0


In [5]:
matchData = cleanLeagueData(matchData)
matchData

,gameDuration,gameMode,gameType,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,...,secondaryPerk0,secondaryVar10,secondaryVar20,secondaryVar30,secondaryPerk1,secondaryVar11,secondaryVar21,secondaryVar31,weekDay,startHourCat
0,1012,CLASSIC,MATCHED_GAME,0,0,0,0,0,0,1,...,8304,12,0,0,8345,3,0,0,2.0,2.0
1,1632,CLASSIC,MATCHED_GAME,0,0,5,0,0,0,0,...,8304,9,4,5,8345,3,0,0,2.0,2.0
2,180,CLASSIC,MATCHED_GAME,0,0,0,0,0,0,0,...,8304,0,0,0,8345,1,0,0,2.0,2.0
3,2146,CLASSIC,MATCHED_GAME,0,0,7,0,0,0,0,...,8304,10,3,0,8345,3,0,0,1.0,2.0
4,2191,CLASSIC,MATCHED_GAME,0,0,8,0,0,0,0,...,8304,10,3,0,8345,3,0,0,0.0,3.0
5,1563,CLASSIC,MATCHED_GAME,0,0,2,0,0,0,6,...,8139,723,0,0,8135,320,4,0,0.0,2.0
6,1341,CLASSIC,MATCHED_GAME,0,0,2,0,0,0,4,...,8304,9,4,5,8345,3,0,0,0.0,2.0
7,1565,ARAM,MATCHED_GAME,0,0,19,0,0,0,0,...,8139,1448,0,0,8135,450,5,0,0.0,2.0
8,1193,CLASSIC,MATCHED_GAME,0,0,8,0,0,0,8,...,8304,9,0,0,8345,3,0,0,0.0,2.0
9,131,CLASSIC,MATCHED_GAME,0,0,0,0,0,0,0,...,8304,0,0,0,8345,1,0,0,0.0,2.0


In [6]:
matchData.loc[:,'weekDay'].value_counts()

0.0    7
2.0    3
1.0    1
Name: weekDay, dtype: int64

In [7]:
matchData.loc[:,'startHourCat'].value_counts()

2.0    10
3.0     1
Name: startHourCat, dtype: int64

In [8]:
matchData.loc[:,['startHourCat', 'weekDay']].value_counts()

startHourCat  weekDay
2.0           0.0        6
              2.0        3
              1.0        1
3.0           0.0        1
dtype: int64

In [9]:
matchData.loc[:,'startHourCat'].mode()

0    2.0
Name: startHourCat, dtype: float64

In [10]:
matchData.loc[:,'weekDay'].mode()

0    0.0
Name: weekDay, dtype: float64